In [1]:
# Install required packages
# !pip install dspy-ai litellm mlflow python-dotenv

import dspy
import mlflow
import os
from dotenv import load_dotenv
from typing import List

print("✅ All imports successful!")

✅ All imports successful!


In [2]:
# Load environment variables
load_dotenv()

# Set up MLflow experiment
mlflow.set_experiment("DSPy-Optimizer-Groq")

# Enable comprehensive tracing for optimization
mlflow.dspy.autolog(
    log_evals=True,           # Log evaluation results
    log_compiles=True,        # Log optimization process
    log_traces_from_compile=True  # Trace during optimization
)

print("🔍 MLflow tracing enabled with optimization tracking!")
print(f"📊 Experiment: DSPy-Optimizer-Groq")


2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/04 14:17:50 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/04 14:17:50 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/04 14:17:50 INFO mlflow.tracking.fluent: Experiment with name 'DSPy-Optimizer-Groq' does not exist. Creating a new experiment.


🔍 MLflow tracing enabled with optimization tracking!
📊 Experiment: DSPy-Optimizer-Groq


In [3]:
# Configure Groq with Llama 3.3 70B Versatile
lm = dspy.LM(
    'groq/llama-3.3-70b-versatile',
    api_key=os.getenv('GROQ_API_KEY'),
    max_tokens=1024,
    temperature=0.7
)

# Set as default LM
dspy.configure(lm=lm)

print("🚀 Groq (Llama 3.3 70B Versatile) configured!")
print(f"Model: {lm.model}")
print(f"📊 Free tier limits:")
print(f"   - Requests: 30/min, 1K/day")
print(f"   - Tokens: 12K/min, 100K/day")

# Quick test
test = dspy.Predict("question -> answer")
result = test(question="What is the capital of France?")
print(f"\nTest: Capital of France = {result.answer}")
print("✅ Groq connection verified!")


🚀 Groq (Llama 3.3 70B Versatile) configured!
Model: groq/llama-3.3-70b-versatile
📊 Free tier limits:
   - Requests: 30/min, 1K/day
   - Tokens: 12K/min, 100K/day

Test: Capital of France = The capital of France is Paris.
✅ Groq connection verified!


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## an...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
